In [3]:
import numpy as np
import random
import matplotlib.pyplot as plt
import tclab
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.model_selection import train_test_split 
import imageio

In [1]:
def generate_temperature_setpoints(initial_rest, initial_temp, min_interval, max_interval, min_temp, max_temp, count, seed=None):
    if seed:
        random.seed(seed)
    start = 0
    end = initial_rest
    TCLab = tclab.setup(connected=False)
    with TCLab() as lab:
        Tsp = np.ones(count) * lab.T1
    while end <= count:
        start = end
        end += random.randint(min_interval, max_interval) 
        Tsp[start:end] = random.randint(min_temp, max_temp)
    return Tsp

In [ ]:
def set_temperature_setpoints(x, y, count):
    Tsp = np.zeros(count)
    for i in range(len(x) - 1):
        Tsp[x[i]:x[i + 1]] = y[i]
    Tsp[x[len(x) - 1]:count] = y[len(y) - 1]
    return Tsp

In [ ]:
def select_best_features(X, y, threshold = 0.05):
    bestfeatures = SelectKBest(score_func=f_regression, k='all')
    fit = bestfeatures.fit(X,y)
    scores = fit.scores_
    total_score = sum(scores)
    
    features = list()
    
    for i in range(len(X.columns)):
        if scores[i] >= threshold * total_score:
            features.append(X.columns[i])
    
    plt.bar(x=X.columns,height=fit.scores_ / total_score);
    
    return X[features]

In [ ]:
def create_train_and_test_data(X, y, s_x, s_y, window, test_size=0.2, shuffle=False):

        Xs = s_x.fit_transform(X)
        ys = s_y.fit_transform(y)

        X_lstm = []
        y_lstm = []
        for i in range(window, len(X)):
            X_lstm.append(Xs[i-window:i])
            y_lstm.append(ys[i])

        X_lstm, y_lstm = np.array(X_lstm), np.array(y_lstm)
        return train_test_split(X_lstm, y_lstm, test_size=test_size, shuffle=shuffle)

In [ ]:
def run_tclab(run_time, Tsp, controllers, loggers=None, connected=True, speedup=1, dt=1, Q2=None):
    
    loops = int(run_time) + 1
    
    T1 = np.zeros(loops)
    Qs = np.zeros([len(controllers), loops])
    tm = np.zeros(loops)
    
    TCLab = tclab.setup(connected=connected, speedup=speedup)
    with TCLab() as lab:
        
        if loggers:
            for logger in loggers:
                logger.log('{:6} {:6} {:6} {:6}'.format('tm', 'Tsp', 'T1', 'Q1'))
        
        last_i = -1
        for clock in tclab.clock(run_time, dt):
            i = int(clock)
            index_gap = i - last_i
            tm[last_i + 1: i + 1] = [clock] * index_gap
            T1[last_i + 1: i + 1] = [lab.T1] * index_gap
            Qs[:, last_i + 1: i + 1] = [[controller.get_output(Tsp[i], T1[i], dt)] * index_gap for controller in controllers]
            lab.Q1(Qs[0, i])
            if Q2:
                lab.Q2(Q2[i])
            last_i = i
            
            if loggers:
                for logger in loggers:
                    logger.log(f'{tm[i]:6.1f} {Tsp[i]:6.2f} {T1[i]:6.2f} {Qs[0, i]:6.2f} {" ".join([f"{x:6.2f}" for x in Qs[1:, i]])}')
            
        return tm, T1, Qs[0], Qs[1:]